In [1]:
from platform import python_version
print(python_version())

3.8.3


# Spame Classifier
***Based on project from O'Reilly's Hands_on Machine Learning with Scikit-Learn and TensorFlow***


In [2]:
import os
import tarfile 
from six.moves import urllib 

URL_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = "http://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham.tar.bz2"
SPAM_URL = "http://spamassassin.apache.org/old/publiccorpus/20030228_spam.tar.bz2"

DIR_ROOT = "Datasets/spamnham"
HAM_DIRNAME = "ham_files"
SPAM_DIRNAME = "spam_files"

In [3]:
def get_files(dir_root=DIR_ROOT):
    if not os.path.isdir(DIR_ROOT):
        os.makedirs(DIR_ROOT)
    for url, filename in ((HAM_URL, HAM_DIRNAME), (SPAM_URL, SPAM_DIRNAME)): 
        path = os.path.join(DIR_ROOT, filename)
        print(path)
        if not os.path.isdir(path):
            urllib.request.urlretrieve(url, path)
        tar = tarfile.open(path)
        tar.extractall(path=DIR_ROOT)
        tar.close()

In [4]:
get_files()

Datasets/spamnham/ham_files
Datasets/spamnham/spam_files


### now need to extract the file into an email object

In [5]:
import email
import email.policy

def parse_emails(path=DIR_ROOT, spam_emails=False):
    pre_ham_list = os.listdir(os.path.join(DIR_ROOT, "easy_ham"))
    pre_spam_list = os.listdir(os.path.join(DIR_ROOT, "spam"))
    ham_list = [email for email in sorted(pre_ham_list) if len(email)>20]
    spam_list = [email for email in sorted(pre_spam_list) if len(email)>20]
    emails = []
    
    if spam_emails: 
        for filename in spam_list:
            with open(os.path.join(DIR_ROOT, "spam", filename), 'rb') as spamfile:
                emails.append(email.parser.BytesParser(policy=email.policy.default).parse(spamfile))
    else:
        for filename in ham_list:
            with open(os.path.join(DIR_ROOT, "easy_ham", filename), 'rb') as hamfile:
                emails.append(email.parser.BytesParser(policy=email.policy.default).parse(hamfile))
                
    return emails

In [6]:
ham_emails = parse_emails()
spam_emails = parse_emails(spam_emails=True)
type(spam_emails[0])

email.message.EmailMessage

In [7]:
len(ham_emails)

2500

In [18]:
from email.iterators import _structure

def get_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()

In [19]:
structs = get_structure(ham_emails[32])
print(structs)

text/plain


In [20]:
type(structs)

str

In [21]:
from collections import Counter

def count_structures(emails):
    count = Counter()
    for email in emails: 
        struct = get_structure(email)
        count[struct] += 1 
    return count

In [22]:
counted = count_structures(spam_emails)
counted


Counter({'text/html': 183,
         'text/plain': 218,
         'multipart(text/plain, application/octet-stream)': 1,
         'multipart(text/html)': 20,
         'multipart(text/plain, text/html)': 45,
         'multipart(text/plain)': 19,
         'multipart(text/html, text/plain)': 1,
         'multipart(text/html, application/octet-stream)': 2,
         'multipart(multipart(text/html))': 5,
         'multipart(text/plain, image/jpeg)': 3,
         'multipart(multipart(text/html), application/octet-stream, image/jpeg)': 1,
         'multipart(multipart(text/plain, text/html), image/gif)': 1,
         'multipart/alternative': 1})

## Split into test and training set

In [23]:
import numpy as np 
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails)
y = np.array([0] * len(ham_emails) +  [1] * len(spam_emails))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

<ipython-input-23-95bc6cc8ee4a>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X = np.array(ham_emails + spam_emails)


In [24]:
from bs4 import BeautifulSoup
def html_to_plain_text(html): 
    soup = BeautifulSoup(html, 'html.parser')

In [25]:


def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [26]:
bad_html = []
for email in spam_emails: 
    if get_structure(email) == "text/html": 
        bad_html.append(email)
len(bad_html)

183

In [27]:
for email in spam_emails: 
    print(type(email_to_text(email)))

<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'NoneType'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<class 'str'>
<class 'NoneType'>
<class 'NoneType'>
<class 'str'>
<class 'NoneType'>
<class 'str'>
<c

In [28]:
print(email_to_text(bad_html[123]))

None


# Create the Pipeline

In [29]:
from sklearn.base import BaseEstimator,TransformerMixin
import nltk
import re
import urlextract

class EmailToWordCountTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True, 
                 replace_urls=True, replace_numbers=True, stemming=True): 
        self.strip_headers=strip_headers
        self.lower_case = lower_case
        self.remove_punctuation=remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers=replace_numbers
        self.stemming = stemming
  
        
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None): 
        X_transformed = []
        for email in X: 
            text = email_to_text(email)
            if self.lower_case: 
                text = text.lower()
            if self.replace_urls: 
                urlextractor = urlextract.URLExtract()
                urls = list(set(urlextractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmer = nltk.PorterStemmer()
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)
                

In [30]:
a_few = X_train[:5]
X_few_wordcounts = EmailToWordCountTransformer().fit_transform(a_few)
X_few_wordcounts

AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10) #returns whichever is bigger count or 10
        most_common = total_count.most_common()[:self.vocabulary_size]#the most common words in order
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)} #word appearences 
        #in most common order. no fequency associated
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X): #row is the enumerate index and word_count the dict of words and 
            # word frequency for each email
            for word, count in word_count.items():
                rows.append(row)# rows become an index for each email
                cols.append(self.vocabulary_.get(word, 0)) #get the count of the word in vocab or 0 if it does not 
                # exist
                data.append(count) #
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [ ]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=100)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors.shape

In [ ]:
X_few_vectors.toarray()

In [ ]:
vocab_transformer.vocabulary_

In [ ]:
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("email_to_wordcount", EmailToWordCountTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_train)